## Environmental Considerations

In [1]:
# !pip install flair
# !pip install tqdm
# !ls -lh


import os
os.chdir("d:\\Thesis\\SemEval15\\SemEval-PIT2015-github\\data")
os.listdir()

['dev.data',
 'SemEval15DataFiles',
 'test.data',
 'test.label',
 'train.data',
 'tweetText.txt',
 'tweetTextFromDevData',
 'tweetTextFromDevData_parsed.txt',
 'tweetTextFromTestData',
 'tweetTextFromTestData_parsed.txt',
 'tweetTextFromTrainData',
 'tweetTextFromTrainData_parsed.txt',
 'tweetText_parsed.txt']

In [2]:
import tqdm
import json
import pandas as pd
import torch
import numpy as np

## Lets load the data in

In [3]:
import urllib.request
import zipfile

baseDir = os.getcwd()
dataDir = baseDir + os.sep + "/SemEval15DataFiles/"
os.makedirs(dataDir, exist_ok=True)
os.chdir(dataDir)

url = 'https://github.com/upmangaurav/t2v/raw/master/data/data.zip'
print('Beginning to download the data file ...')
urllib.request.urlretrieve(url, dataDir + 'data.zip')

zip_ref = zipfile.ZipFile('data.zip', 'r')
zip_ref.extractall(dataDir)
print('Extracted downloaded data file...')
zip_ref.close()

!ls -lh
os.listdir()

Beginning to download the data file ...
Extracted downloaded data file...


'ls' is not recognized as an internal or external command,
operable program or batch file.


['data.zip', 'dev.data', 'test.data', 'test.label', 'train.data']

In [4]:
def make_List(file):
    debatableCount = 0
    with open(file, 'r') as f:
        td = f.readlines()

    dataList = []
    print("\nSize of Dataset", file, str(len(td)))
    for item in td:
        trainDict = {}
        splitsie = item.split('\t')

        #For training and dev data:
        if len(splitsie[4]) > 1:
            #debatable if only 2 turkers voted similar
            if splitsie[4][1] == '2':
                debatableCount += 1
                continue
        #Convert Label such as (3, 2) to decimal value like 0.6
            else: trainDict['Label'] = 0.2 * int(splitsie[4][1])

        else: # Test data:
            if splitsie[4] == '2':
                debatableCount += 1
                continue

            else: trainDict['Label'] = 0.2 * int(splitsie[4])

        
        trainDict['Topic_Id'] = splitsie[0]
        trainDict['Topic_Name'] = splitsie[1]
        trainDict['Sent_1'] = splitsie[2]
        trainDict['Sent_2'] = splitsie[3]


        dataList.append(trainDict)
    
    print("Debatable thus ignored tweet-combo count: ", debatableCount)
    print("Final dataset size:", len(dataList))
    return dataList 

## Data Exploration

In [5]:
trainList = make_List('train.data')
testList = make_List('test.data')
devList = make_List('dev.data')
trainList[:5]


Size of Dataset train.data 13063
Debatable thus ignored tweet-combo count:  1533
Final dataset size: 11530

Size of Dataset test.data 972
Debatable thus ignored tweet-combo count:  130
Final dataset size: 842

Size of Dataset dev.data 4727
Debatable thus ignored tweet-combo count:  585
Final dataset size: 4142


[{'Label': 1.0,
  'Topic_Id': '4',
  'Topic_Name': '1st QB',
  'Sent_1': 'EJ Manuel the 1st QB to go in this draft',
  'Sent_2': 'But my bro from the 757 EJ Manuel is the 1st QB gone'},
 {'Label': 1.0,
  'Topic_Id': '4',
  'Topic_Name': '1st QB',
  'Sent_1': 'EJ Manuel the 1st QB to go in this draft',
  'Sent_2': 'Can believe EJ Manuel went as the 1st QB in the draft'},
 {'Label': 0.6000000000000001,
  'Topic_Id': '4',
  'Topic_Name': '1st QB',
  'Sent_1': 'EJ Manuel the 1st QB to go in this draft',
  'Sent_2': 'EJ MANUEL IS THE 1ST QB what'},
 {'Label': 0.8,
  'Topic_Id': '4',
  'Topic_Name': '1st QB',
  'Sent_1': 'EJ Manuel the 1st QB to go in this draft',
  'Sent_2': 'Manuel is the 1st QB to get drafted'},
 {'Label': 1.0,
  'Topic_Id': '4',
  'Topic_Name': '1st QB',
  'Sent_1': 'EJ Manuel the 1st QB to go in this draft',
  'Sent_2': 'My boy EJ Manuel being the 1st QB picked'}]

# Initiate Embeddings

In [6]:
print(torch.cuda.is_available())
torch.cuda.get_device_name(0)

True


'GeForce GTX 1050 Ti'

In [7]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, DocumentPoolEmbeddings, DocumentRNNEmbeddings, Sentence
#, ELMoEmbeddings, BertEmbeddings
# BytePairEmbeddings

#BytePair Embeddings
# BytePairEmbedding = BytePairEmbeddings('en')

# initialize the word embeddings
glove_embedding = WordEmbeddings('glove')
flair_embedding_forward = FlairEmbeddings('mix-forward')
flair_embedding_backward = FlairEmbeddings('mix-backward')
# bert_embedding = BertEmbeddings('bert-large-cased')
# elmo_embedding = ELMoEmbeddings()

embeddings = [
#                                               elmo_embedding
#                                               BytePairEmbedding, 
                                              glove_embedding,
#                                               flair_embedding_backward,
#                                               flair_embedding_forward,
#                                              bert_embedding
] 

# initialize the document embeddings, mode = mean
document_embeddings = DocumentRNNEmbeddings(embeddings)

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [8]:
# Some HIMYM sentence
sentence = Sentence('Lets go to the mall, Today! - Robin Sparkles')
# embed words in sentence 
document_embeddings.embed(sentence)
for token in sentence:
    print(token.embedding)
# data type and size of embedding 
print(type(token.embedding))
# storing size (length) *2 because there'll be concatenation of diff and mult vectors
n = token.embedding.size()[0] * 2
n

tensor([-3.5214e-01,  4.0450e-02,  6.9801e-01, -1.4999e-01,  2.9095e-01,
         6.0819e-01, -7.0817e-01,  6.2955e-02,  8.3700e-01, -7.0492e-01,
         5.6426e-01, -9.5148e-02,  1.2830e-02, -5.3448e-01,  1.9840e-01,
         5.9018e-01,  1.8226e-01,  4.3688e-01, -7.5381e-02,  7.8457e-01,
        -5.8933e-01, -6.5748e-01,  7.3637e-02, -1.4729e-01,  5.4231e-01,
        -2.7395e-02, -6.7368e-01, -5.0288e-01,  1.4738e-01, -3.6519e-01,
        -3.7521e-03,  1.5221e+00, -4.9132e-01,  7.2426e-02,  9.7861e-02,
         2.5034e-01, -1.8353e-01, -6.1457e-01,  5.3440e-01, -4.9145e-01,
         1.0121e-01, -1.8739e-02, -3.6560e-01, -2.6097e-01, -8.7821e-01,
         1.0591e-01, -5.6150e-01, -2.0758e-01,  5.3582e-01,  1.0921e-01,
        -1.2925e-01,  7.0651e-02, -1.5121e-03,  1.2543e-01,  2.6603e-01,
        -1.1057e+00, -1.5336e-01, -6.6789e-02,  1.5032e+00, -2.9650e-01,
         3.8293e-01,  2.2578e-01,  5.3931e-01,  5.2797e-02,  2.5743e-01,
         6.3961e-01,  5.2151e-01,  2.3681e-01, -1.0

        -0.1594, -0.0536, -0.4989,  0.6593])
<class 'torch.Tensor'>


200

In [9]:
def generateBinaryLabels(dataList, binaryOrNot):
    if binaryOrNot:
        dataLabels = []
        for couple in dataList:
            if couple['Label'] >= 0.6:
                dataLabels.append(1)
            elif couple['Label'] <= 0.2:
                dataLabels.append(0)
            else:
                print(couple['Label'])
        print("No of labels: ", len(dataLabels))
    return dataLabels

In [10]:
def generateEmbeddings(someList):
    from tqdm import tqdm
    n = sentence.embedding.size()[0] * 2
    s = torch.zeros(0,n)
    for i in tqdm(range(len(someList))):

        # retrieve the text sentence
        sentence1 = Sentence(someList[i]['Sent_1'])
        sentence2 = Sentence(someList[i]['Sent_2'])

        # embed the sentences with our document embedding
        document_embeddings.embed(sentence1)
        document_embeddings.embed(sentence2)

        # Calculate the element-wise product
        productTensor = sentence1.get_embedding() * sentence2.get_embedding()
        # Calculate the difference
        absDiffTensor = torch.abs(sentence1.get_embedding() - sentence2.get_embedding())

        # Add the concatenated vector as final embedding
        embeddingVector = torch.cat([productTensor, absDiffTensor])
        
        # Adding Document embeddings to list #
        s = torch.cat((s, embeddingVector.view(-1,n)),0)
    
    return s

In [11]:
import matplotlib.pyplot as plt

trainLabels = generateBinaryLabels(trainList, True)
devLabels = generateBinaryLabels(devList, True)
testLabels = generateBinaryLabels(testList, True)
# pd.DataFrame(trainLabels)[0].value_counts()

N = 5
menMeans = (20, 35, 30, 35, 27)
womenMeans = (25, 32, 34, 20, 25)
ind = np.arange(N)    # the x locations for the groups
width = 0.35       # the width of the bars: can also be len(x) sequence

p1 = plt.bar(ind, menMeans, width)
p2 = plt.bar(ind, womenMeans, width,
             bottom=menMeans)

# plt.show()

No of labels:  11530
No of labels:  4142
No of labels:  842


In [12]:
if int(trainList[4]['Label']) >= 0.6:
    print("yeah")
trainLabels[4]

yeah


1

In [13]:
trainList = make_List('train.data')
trainEmbeddings = generateEmbeddings(trainList)
print(trainEmbeddings.shape)


Size of Dataset train.data 13063
Debatable thus ignored tweet-combo count:  1533
Final dataset size: 11530


 18%|██████████████                                                              | 2131/11530 [00:14<01:10, 134.27it/s]

RuntimeError: CUDA out of memory. Tried to allocate 1024.00 KiB (GPU 0; 4.00 GiB total capacity; 1.71 GiB already allocated; 0 bytes free; 597.38 MiB cached)

In [ ]:
testList = make_List('test.data')
testEmbeddings = generateEmbeddings(testList)
print(testEmbeddings.shape)

In [ ]:
devList = make_List('dev.data')
devEmbeddings = generateEmbeddings(devList)
print(devEmbeddings.shape)

# Training begins

In [ ]:
binThresHold = 0.075
def custom_eval(preds, dtrain):
    labels = dtrain.get_label().astype(np.int)
    preds = (preds >= 0.075).astype(np.int)
    return [('f1_score', f1_score(labels, preds))]

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

x_train = trainEmbeddings.numpy()
y_train = np.array(trainLabels).reshape(len(trainLabels), -1)
x_valid = devEmbeddings.numpy()
y_valid = np.array(devLabels).reshape(len(devLabels), -1)

### XGBoost compatible data ###
dtrain = xgb.DMatrix(x_train,y_train)
dvalid = xgb.DMatrix(x_valid, label = y_valid)

### defining parameters ###
params = {
          'colsample': 0.9,
          'colsample_bytree': 0.5,
          'eta': 0.1,
          'max_depth': 8,
          'min_child_weight': 6,
          'objective': 'binary:logistic',
          'subsample': 0.9
          }

### Training the model ###
xgb_model = xgb.train(
                      params,
                      dtrain,
                      feval= custom_eval,
                      num_boost_round= 1000,
                      maximize=True,
                      evals=[(dvalid, "Validation")],
                      early_stopping_rounds=30
                      )


## Testing

In [ ]:
### Reformatting test set for XGB ###
dtest = xgb.DMatrix(devEmbeddings)

### Predicting ###
predict = xgb_model.predict(dtest) # predicting

In [ ]:
print(predict[198])
print(devLabels[198])

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

y_pred = np.array([1 if (predict[i] >= binThresHold) else 0 for i in range(len(predict))])
y_true = np.array(devLabels)
score = precision_recall_fscore_support(y_true, y_pred, average='macro')

In [ ]:
score

 18%|██████████████                                                              | 2131/11530 [00:30<01:10, 134.27it/s]